In [4]:
# imports
import json
import string
import pandas as pd
import numpy as np
from transformers import pipeline
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os
from tqdm import tqdm

/Users/pirapuraj/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tokenizer = AutoTokenizer.from_pretrained("Paraon/flan-t5-xl-clickbait-spoilering")
model = AutoModelForSeq2SeqLM.from_pretrained("Paraon/flan-t5-xl-clickbait-spoilering")

Loading checkpoint shards: 100%|██████████| 3/3 [00:16<00:00,  5.36s/it]


In [6]:
test_set_path = "./datasets"
test_set = os.path.join(test_set_path, 'test.jsonl')

In [7]:
data = pd.read_json(test_set, lines=True)
print("Total samples in test set: ", len(data))


Total samples in test set:  1000


In [8]:
# Load the clickbait article text
para = data['targetParagraphs'].iloc[0]
spoilers = data["spoiler"].iloc[0]
# para = data['targetParagraphs']
# spoilers = data["spoiler"]


In [9]:
paragraphs = data['targetParagraphs'][:150]
spoilers = data['spoiler'][:150]
post_text = data['postText'][:150]

print("paragraph: ", paragraphs)
print("postText: ", post_text)
print("Spoiler: ", spoilers)
true_spoilers = []
predicted_spoilers = []

# print(paragraphs)
# for pr, sp in zip(paragraphs, spoilers):
#     pra = " ".join(pr)
#     spa = " ".join(sp)
#     print("paragraph: ", pra)
#     print("Spoiler: ", spa)


# paragraphs_joined = " ".join(paragraphs)
# spoilers_joined = " ".join(spoilers)
# # print(paragraphs_joined)

# paragraphs_to_check = [] 
# paragraphs_to_check.append(paragraphs_joined)

# spoilers_to_check = [] 
# spoilers_to_check.append(spoilers_joined)

#base_prompt = "Extract very important point only from the below paragraph:\n"
base_prompt = "spoler is very important Extract spoiler points from the below paragraph:\n "
#base_prompt = "Extract spoilers from the below paragraph:\n"

for pra, sp, pst in zip(paragraphs, spoilers, post_text):
    para = " ".join(pra)
    spoil = " ".join(sp)
    post = " ".join(pst)

    print("paragraph: ", para)
    print("postText: ", post)
    print("Spoiler: ", spoil)
   
    #print(para)
    article_text = para
    spoiler = spoil

    true_spoilers.append(spoiler)
    
    # print("Target paragraph: ", article_text)

    # print("Length of string: ", len(article_text))

    # Prompt for LLM
    prompt =  base_prompt + "Paragraph: " + '"' + article_text + '"'

    # print(prompt)

    inputs = tokenizer.encode(prompt, return_tensors="pt")

    # Extract Spoilers
    outputs = model.generate(inputs, max_length=100, early_stopping=True)

    # Generated sequences
    spoilers = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    print("True spoiler: ", spoiler)
    print("Posted Text: ", pst)

    # Print the extracted spoilers
    for i, spoiler in enumerate(spoilers):
        print(f"Predicted Spoiler {i+1}: {spoiler}")
        predicted_spoilers.append(spoiler)

Token indices sequence length is longer than the specified maximum sequence length for this model (1204 > 512). Running this sequence through the model will result in indexing errors


paragraph:  0      [Erin Zaikis often used to think about how she...
1      [We weren't expecting this one!, On Wednesday,...
2      [This whole Taylor Swift, Tom Hiddleston thing...
3      [While spending his summer in Los Angeles, Den...
4      [I believe that the cute does go on., 1. Remem...
                             ...                        
145    [Winter is only months away, leaving many of u...
146    [Tennessee could become the 21st state in the ...
147    [A healthy 20-year-old woman was running on a ...
148    [Tesla is gearing up to release its long-await...
149    [Pebble on Tuesday announced its first non-sma...
Name: targetParagraphs, Length: 150, dtype: object
postText:  0               [This simple household item saves lives]
1      [You'll Never Believe Who Robert Downey Jr.'s ...
2      [THE NEWEST THEORY ON HIDDLESWIFT'S ROMANCE IS...
3      [NBA Star surprises an inner-city youth basket...
4      [The little girl from "Titanic" is all grown u...
              

/Users/pirapuraj/miniconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


True spoiler:  soap
Posted Text:  ['This simple household item saves lives']
Predicted Spoiler 1: Erin Zaikis. "The first soap burnt all my bowls and dishes," she laughed. "It was a really horrible couple of months," she said. "It was just a fun one to wear."
paragraph:  We weren't expecting this one! On Wednesday, Robert Downey Jr. visited the Howard Stern to drop a bomb about his "free pass". Even though the actor is happily married to his wife Susan Downey since 2005, he as an unspoken agreement to mack on a certain actress if the opportunity arises. Opening up to Howard about his lucky lady, the actor admitted the woman in question is actually a good friend of Susan's. Uh, awkward much?! So, which leading lady does the poppa want to get it on with? According to R.D.J., the object of his affection is none other than Gwyneth Paltrow! Elaborating on his surprising crush, Robert confessed: "My free pass is, because her and Susan are such good friends, is Paltrow. I gotta get her back i

KeyboardInterrupt: 

In [ ]:
# Write true spoilers to file
with open("./output/true_spoiler.txt", "w") as f:
    for ts in true_spoilers:
        f.write(ts + '\n')

In [ ]:
# Write predicted spoilers to file
with open("./output/predicted_spoiler.txt", "w") as f:
    for ps in predicted_spoilers:
        f.write(ps + '\n')
    

In [10]:
!bash test_scores.sh

{
 "name": "BLEU",
 "score": 16.9,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.2",
 "verbose_score": "23.8/17.2/14.9/13.2 (BP = 1.000 ratio = 1.639 hyp_len = 12318 ref_len = 7514)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.2"
}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
